# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "../Weatherpy/weather_df.csv"
# output_data_file

types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df

,City,Cloudiness,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Country
0,punta arenas,32,1587176943,61,8.99,125.34,88.45,2.59,PH
1,upernavik,98,1587176903,90,72.79,-56.15,21.58,4.09,GL
2,mataura,35,1587176907,54,-46.19,168.86,60.01,11.99,NZ
3,lichinga,32,1587176944,71,-13.31,35.24,59.65,5.50,MZ
4,saskylakh,0,1587176923,88,71.92,114.08,8.71,4.99,RU
...,...,...,...,...,...,...,...,...,...
530,ukhta,100,1587177027,94,63.57,53.68,34.54,6.64,RU
531,kutum,0,1587176958,20,14.20,24.67,63.99,8.37,SD
532,barguzin,83,1587177028,57,53.62,109.62,39.63,10.74,RU
533,gondar,1,1587177028,14,12.60,37.47,62.92,3.67,ET


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data_df[["Lat", "Lng"]]
# locations

humidity = weather_data_df["Humidity"].astype(float)

# humidity

# figure_layout = {
#     'width' : '1000px',
#     'height':'400px',
#     'border':'1px solid black',
#     'padding':'1px',
#     'margin':'0 auto 0 auto'
# }

# fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
75,morondava,74.70,8.81,0
76,saint-leu,71.60,9.17,0
186,saint-pierre,71.60,9.17,0
433,panama city,75.99,1.99,0
483,flinders,72.00,1.01,0
516,sur,79.23,6.15,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,punta arenas,PH,8.99,125.34,
1,upernavik,GL,72.79,-56.15,
2,mataura,NZ,-46.19,168.86,
3,lichinga,MZ,-13.31,35.24,
4,saskylakh,RU,71.92,114.08,
...,...,...,...,...,...
530,ukhta,RU,63.57,53.68,
531,kutum,SD,14.20,24.67,
532,barguzin,RU,53.62,109.62,
533,gondar,ET,12.60,37.47,


In [7]:
# coordinates_lat_lng = []

# target_city = hotel_df["City"]
# # target_city

# target_type = "hotel"

# hotel_df["Coordinates"] = hotel_df["Lat"] + "," + hotel_df["Lng"]

# radius = 5000

# params = {
#     "location:" : coordinates_lat_lng,
#     "types" : target_type,
#     "radius" : radius, 
#     "key" : g_key
# }

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# print("A HOTEL WITHIN A 5000 METER RADIUS")

# response = requests.get(base_url, params)
# # print(response.url)

# hotel_distance = response.json()
# print(json.dumps(hotel_distance, indent=4, sort_keys=True))

target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [8]:
# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params = {
#     "location": (hotel_df["Lat"], hotel_df["Lng"]),
#     "rankby": "distance",
#     "type" : "hotel", 
#     "key" : g_key
# }

# # params
pprint(response.json(), depth=1)

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


In [9]:
hotel_df.iloc[0][0]

'punta arenas'

In [10]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
narrowed_city_df = hotel_df

In [11]:
hotel_name

[nan,
 'Café de Upernavik',
 "Ellie's Villa",
 'Girassol Lichinga',
 nan,
 'Kak Doma',
 nan,
 'The Explorer Hotel',
 "Pension Maro'i",
 'Place to Sleep Hotel Lohja',
 nan,
 nan,
 'Best Western Danbury/Bethel',
 'The Tukad Villa',
 nan,
 'Raqui Lodge',
 nan,
 'Radisson Blu Polar Hotel, Spitsbergen',
 'Hostal Las Lilas',
 nan,
 'Hilton Garden Inn Sacramento Elk Grove',
 'Ungoofaaru Inn',
 "Pinpin d'Amour",
 'Capoia´s Hotel',
 'Sands Hotel Margate',
 'Mercure Bengkulu',
 nan,
 'Nusa Island Retreat',
 'Quinta Samay',
 'Birkenhead House',
 'Wingate by Wyndham Alpharetta',
 nan,
 'Sunshine Hotel',
 'Gostinitsa Uyut',
 'Broadwater Resort',
 'Shelikof Lodge',
 'Top of the World Hotel',
 'Grand Naniloa Hotel Hilo - a DoubleTree by Hilton',
 'Hotel Buganville Express',
 'City Gate Hotel and Restaurant',
 'Pebbles Inn',
 nan,
 nan,
 'Bem-Vindo Lodge',
 nan,
 'Hostel Castle Tirano / BB',
 'Bredasdorp Country Manor',
 'Nomades Hotel Boutique',
 nan,
 'Auberge de Saint-Aubin',
 'Rede Beach: Hotel em

In [12]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))